# Create Vector Databse with Embedding from Research Papers

Define directories

In [1]:
import os

base_dir = os.getcwd()
sample_pdf_dir = os.path.join(base_dir, 'sample_pdfs')
persist_dir = os.path.join(base_dir, 'database')

Load PDF into arrays

In [2]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

In [22]:
import sys
print(sys.path)
sys.path.append('/Users/ruigekong/anaconda3/lib/python3.11/site-packages')

['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/Users/ruigekong/PycharmProjects/2023BioxAIHackathon/AgentExperimentation', '/Users/ruigekong/PycharmProjects/2023BioxAIHackathon', '/Users/ruigekong/anaconda3/lib/python311.zip', '/Users/ruigekong/anaconda3/lib/python3.11', '/Users/ruigekong/anaconda3/lib/python3.11/lib-dynload', '', '/Users/ruigekong/anaconda3/lib/python3.11/site-packages', '/Users/ruigekong/anaconda3/lib/python3.11/site-packages/aeosa', '/Users/ruigekong/anaconda3/lib/python3.11/site-packages', '/Users/ruigekong/anaconda3/lib/python3.11/site-packages']


In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

embedding = OpenAIEmbeddings()
loader = PyPDFLoader(os.path.join(sample_pdf_dir,'combined_research_pdf_sample.pdf'),extract_images=False)
pdfs = loader.load()
print(f'page 0: {pdfs[0].page_content}')
db = Chroma.from_documents(documents=pdfs,
                           embedding=embedding,
                           persist_directory=persist_dir)

page 0: Designing your Analysis 
By Tallulah Andrews 


AuthenticationError: Incorrect API key provided: 1. You can find your API key at https://platform.openai.com/account/api-keys.

We can then persist the database to disk

In [24]:
db.persist()
db = None

And we can load the persisted databse from disk every time

In [31]:
db = Chroma(persist_directory=persist_dir,
                  embedding_function=embedding)

Example Query

In [7]:
query = "What is High-throughput single-cell transcriptomics on organoids?"
docs = db.similarity_search(query)
print(docs[0].page_content)

Exercise Answers: (2) 
How do different Huntington gene variants affect neuronal identity/function? 
●What type of question is being asked? 
Comparative 
●What data do you need to answer the question? 
Single cell RNAseq from brain samples or brain organoids from multiple samples carrying 
different Huntington gene variants. 
(Bulk RNAseq of sorted neurons is also valid) 


Now, we use the openai embeddings. Running the cell below would prompt us to enter the api key.

# Create retriever

In [32]:
retriever = db.as_retriever()

# Create a question answering chain

In [33]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name='gpt-4',
    temperature=0.1
)

# Create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True,
                                  verbose=True)

In [34]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])


In [35]:
# Question
query = "What is an organoid?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new RetrievalQA chain...

> Finished chain.
An organoid is a miniaturized and simplified version of an organ produced in vitro in three dimensions that shows realistic micro-anatomy. They are derived from one or a few cells from a tissue, embryonic stem cells or induced pluripotent stem cells, which can self-organize in three-dimensional culture owing to their self-renewal and differentiation capacities.


Sources:
/Users/ruigekong/PycharmProjects/2023BioxAIHackathon/AgentExperimentation/BioinformaticsPDF/combined.pdf


In [1]:
import agents
from agents import BioinformaticsAgent

bioinformatics_agent = BioinformaticsAgent()
print(bioinformatics_agent)

In [5]:
query = 'What are you?'

bioinformatics_agent.invoke(query)

'The text does not provide information on the role of bioinformatics in drug discovery.'